# RESnet50 transfer learning

## 1. Import libraries and load data

In [1]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = ""
#os.environ["AUTOGRAPH_VERBOSITY"] = "10"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"
from tqdm import tqdm 
import time 
from platform import python_version
import warnings
import time
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import multiprocessing as mp
import shutil
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import ResNet50, decode_predictions, preprocess_input
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
from tensorflow_addons.layers import WeightNormalization
from sklearn.metrics import classification_report


from wandb.keras import WandbCallback
import pandas as pd
import numpy as np
import seaborn as sn

from PIL import Image
import xml.etree.ElementTree as ET
import psutil
import random

warnings.filterwarnings("ignore")
%matplotlib inline

print("py", python_version())
print("tf", tf.__version__)
print("keras", tf.keras.__version__)
mem = psutil.virtual_memory()
print("mem", mem.total/1024/1024)
cpu = mp.cpu_count()
print("cpu", cpu)

%system nvidia-smi
#%system rocm-smi

py 3.7.6
tf 2.2.0
keras 2.3.0-tf
mem 16045.11328125
cpu 2


['Fri Jul 24 15:16:27 2020       ',
 '+-----------------------------------------------------------------------------+',
 '| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |',
 '|-------------------------------+----------------------+----------------------+',
 '| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |',
 '| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |',
 '|===============================+======================+======================|',
 '|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |',
 '| N/A   38C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |',
 '+-------------------------------+----------------------+----------------------+',
 '                                                                               ',
 '+-----------------------------------------------------------------------------+',
 '| Processes:                          

In [2]:
data_dir = '../input/stanford-dogs-dataset/images/Images'
annotations_dir = '../input/stanford-dogs-dataset/annotations/Annotation' 

## 1.1 variables

In [3]:
breed_list = os.listdir(data_dir)
print("num. breeds total:", len(breed_list))


# Print out the classes we need to target
class_names = [breed.split('-',1)[1] for breed in breed_list] #visualize breeds
print(class_names[:10])

num. breeds total: 120
['Weimaraner', 'affenpinscher', 'Chesapeake_Bay_retriever', 'Sussex_spaniel', 'Leonberg', 'Maltese_dog', 'Saluki', 'kelpie', 'keeshond', 'American_Staffordshire_terrier']


# 2. Image preprocessing

### 2.1 Load data

In [4]:
num_of_categories = 120
image_size = 192
batch_size = 16

In [5]:
num_breeds = 120 # integer between 2 and 120
breeds = breed_list[:num_breeds]

def load_images_and_labels(breeds):
    img_lst=[]
    labels=[]
    
    for index, breed in tqdm(enumerate(breeds)):
        for image_name in os.listdir(data_dir+"/"+breed):
            img = cv2.imread(data_dir+"/"+breed+"/"+image_name)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                       
            img_array = Image.fromarray(img, 'RGB')
            
            #resize image to 224 x 224 because the input image resolution for ResNet50 is 224 x 224
            resized_img = img_array.resize((224, 224))
            
            img_lst.append(np.array(resized_img)) # append image as numpy array
            
            labels.append(class_names[index])
            
    return img_lst, labels 

images, labels = load_images_and_labels(breeds)
print("No. of images loaded = ",len(images),"\nNo. of labels loaded = ",len(labels))

120it [03:02,  1.52s/it]

No. of images loaded =  20580 
No. of labels loaded =  20580


### 2.2 label encoding

In [6]:
# replace numbers with names
le = LabelEncoder()
nlabels = le.fit_transform(labels) # encode labels as number values. This prepares for categorical encoding
Y=to_categorical(nlabels,num_classes = num_breeds) # category encoding

In [7]:
X = np.array(images)

### 2.3 test and train sets 

In [8]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=1)

print("x_train shape = ",x_train.shape)
print("y_train shape = ",y_train.shape)

print("\nx_val shape = ",x_val.shape)
print("y_val shape = ",y_val.shape)

print("\nx_test shape = ",x_test.shape)
print("y_test shape = ",y_test.shape)

x_train shape =  (14817, 224, 224, 3)
y_train shape =  (14817, 120)

x_val shape =  (1647, 224, 224, 3)
y_val shape =  (1647, 120)

x_test shape =  (4116, 224, 224, 3)
y_test shape =  (4116, 120)


### 2.4 features preprocess

In [9]:
# We should preprocess the images the same way resnet images were preprocessed
aug = ImageDataGenerator(rotation_range=30, #rotations (as seen above)
                        width_shift_range=0.2,  # randomly shift images horizontally 
                        height_shift_range=0.2,# randomly shift images vertically 
                        shear_range=0.2, # shear image
                        zoom_range=0.2, # zoom into image 
                        horizontal_flip=True, # randomly flip images
                        rescale=1/255., #rescale array values
                        preprocessing_function=preprocess_input) #  creates a ‘reflection’ and fills the empty values in reverse order of the known values



In [11]:
# Load ResNet50 Trained on imagenet
resnet_model = ResNet50(weights="imagenet")


102973440/102967424 [==============================] - 1s 0us/step


In [10]:
# Build a new model that is ResNet50 minus the very last layer
last_layer = resnet_model.get_layer("avg_pool")

resnet_layers = keras.Model(inputs=resnet_model.inputs, outputs=last_layer.output)


NameError: name 'resnet_model' is not defined

In [ ]:
# We use our resnet to "predict" but because we have removed the top layer, 
# this outputs the activations of the second to last layer on our dataset

x_train_features = resnet_layers.predict(x_train_preprocessed)


x_test_features = resnet_layers.predict(x_test_preprocessed)



# Define new top layers and compile model

In [ ]:
feature_model=Sequential()
feature_model.add(Dense(120, activation="sigmoid"))
feature_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
wandb.init(project="transfer learn")
feature_model.fit(x_train_features, y_train, epochs=50, validation_data=(x_test_features, y_test), callbacks=[WandbCallback()])

In [ ]:
optimizer = Adam(lr=learning_rate)
# optimizer = RMSprop(lr=learning_rate)

loss = "categorical_crossentropy"
# loss = "kullback_leibler_divergence"

for layer in model.layers:
    layer.trainable = True

model.compile(optimizer=optimizer,
              loss=loss,
              metrics=["accuracy"])

In [ ]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)

print(test_loss,test_accuracy)

# Fit model

In [ ]:
%%time

history_1 = model.fit_generator(generator=train_generator, 
                                steps_per_epoch=len(train_generator), 
                                validation_data=test_generator, 
                                validation_steps=len(test_generator),
                                epochs=epochs,
                                callbacks=[reducelr, earlystop, lambdacb, tensorboard, checkpoint])

# Training and test loss/accuracy graphs

In [ ]:
plt.figure(figsize=(18, 6))

plt.subplot(121)
loss = history_1.history['loss']
val_loss = history_1.history['val_loss']
plt.plot(loss,"--", linewidth=3 , label="train")
plt.plot(val_loss, linewidth=3 , label="valid")

plt.legend(['train','validation'], loc='upper left')
plt.grid()
plt.ylabel('loss')
plt.ylim((1.5, 3))
plt.xlabel('Epoch')
plt.title('4 layers CNN Model Loss')
plt.legend(['train','validation'], loc='upper left')

plt.subplot(122)
acc = history_1.history['accuracy']
val_acc = history_1.history['val_accuracy']

plt.plot(acc,"--", linewidth=3 , label="train")
plt.plot(val_acc, linewidth=3 , label="valid")

plt.legend(['train','validation'], loc='upper left')
plt.grid()

plt.ylabel('accuracy')
plt.xlabel('Epoch')
plt.title('4 layers CNN Model accuracy')
plt.legend(['train','validation'], loc='upper left')
plt.show()

# Sample prediction

In [ ]:
# Randomly test an image from the test set

# model.load_weights('dog_breed_classifier.h5')

imageno=np.random.random_integers(low=0, high=test_generator.samples)

name = test_generator.filepaths[imageno]
print(name)
plt.imshow(mpimg.imread(name))

img = Image.open(test_generator.filepaths[imageno]).resize((targetx, targety))
probabilities = model.predict(preprocess_input(np.expand_dims(img, axis=0)))
breed_list = tuple(zip(test_generator.class_indices.values(), test_generator.class_indices.keys()))

for i in probabilities[0].argsort()[-5:][::-1]: 
    print(probabilities[0][i], "  :  " , breed_list[i])